In [7]:
import os, uuid, shutil

from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Set connection string
connect_str = "DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName=ipmdsa;AccountKey=Fs9azVVoHgcBHn1wr78+5LPe/UuKCH2+oLiIgvrmrgyhYoZxNS0Hjt8uIR54XUPNaBRH7SNTDSgc+AStvUN2ng==;BlobEndpoint=https://ipmdsa.blob.core.windows.net/;FileEndpoint=https://ipmdsa.file.core.windows.net/;QueueEndpoint=https://ipmdsa.queue.core.windows.net/;TableEndpoint=https://ipmdsa.table.core.windows.net/"

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [8]:
# Create container
container_name = "ipmdcontainer"
try:
    container_client = blob_service_client.create_container(container_name)
except:
    print("El contenedor no se puede crear (ya existe)")    
    container_client = blob_service_client.get_container_client(container_name)

El contenedor no se puede crear (ya existe)


In [9]:
# Create a local directory to hold blob data
local_path = "./data"
os.mkdir(local_path)

# Create a file in the local data directory to upload and download
local_file_name = str(uuid.uuid4()) + ".txt"
upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
file = open(file=upload_file_path, mode='w')
file.write("Hello, World!")
file.close()

# Create a blob client using the local file name as the name for the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

# Upload the created file
with open(file=upload_file_path, mode="rb") as data:
    blob_client.upload_blob(data)


Uploading to Azure Storage as blob:
	20ee9e78-c0da-4580-a99d-c29d44cf0fd0.txt


In [10]:
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)


Listing blobs...
	20ee9e78-c0da-4580-a99d-c29d44cf0fd0.txt


In [11]:
# Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extension so you can see both files in the data directory
download_file_path = os.path.join(local_path, str.replace(local_file_name ,'.txt', 'DOWNLOAD.txt'))
container_client = blob_service_client.get_container_client(container=container_name) 
print("\nDownloading blob to \n\t" + download_file_path)

with open(file=download_file_path, mode="wb") as download_file:
 download_file.write(container_client.download_blob(blob.name).readall())


	./data/20ee9e78-c0da-4580-a99d-c29d44cf0fd0DOWNLOAD.txt


In [12]:
# Clean up

print("Deleting the local source and downloaded files...")
os.remove(upload_file_path)
os.remove(download_file_path)
shutil.rmtree(local_path, ignore_errors=True)

print("Done")

Deleting the local source and downloaded files...
Done


In [13]:
print("Deleting the blob...")
container_client.delete_blob(blob.name)
print("Done")

Deleting the blob...
Done


In [14]:
print("Deleting blob container...")
container_client.delete_container()

Deleting blob container...
